In [ ]:
import mease_lab_to_nwb
from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO
from sonpy import lib as sonpy
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
file_recording = "tests/data/H5_Mech_1Hz_10Hz.smrx"
# file_recording="/mnt/sds/PainData/Corrected_Channel_Map/L6/Cortex/16.12.20/Dual_S1_VPL_Freq_10mW.smrx"
# file_recording="/mnt/sds/Yulia/m6/KS3/P1_S1HL_MechLaser_4mW.smrx"

In [ ]:
plt.rcParams["figure.figsize"] = (30, 3)

In [ ]:
def plot(filename, channel_index, start=0.0, end=184.9, rescale=True):
    spFile = sonpy.SonFile(file_recording, True)
    assert (
        spFile.ChannelType(channel_index) == sonpy.DataType.Adc
    ), "Channel type must be Adc (integer wave)"
    maxTick = spFile.ChannelMaxTime(channel_index)
    print(channel_index, maxTick)
    intervalInTicks = spFile.ChannelDivide(channel_index)
    tickLength = spFile.GetTimeBase()
    maxTime = maxTick * tickLength
    print(maxTick / intervalInTicks)
    data = sonpy.SonFile.ReadInts(
        spFile, channel_index, int(maxTick / intervalInTicks), 0
    )
    magicNumberFromSonpyPdfDocs = 6553.6
    scaleFactor = spFile.GetChannelScale(channel_index) / magicNumberFromSonpyPdfDocs
    if rescale:
        data = data * scaleFactor
        data += spFile.GetChannelOffset(channel_index)
    d_max = np.max(data)
    d_min = np.min(data)
    i_start = int(start / tickLength / intervalInTicks)
    i_end = int(end / tickLength / intervalInTicks)
    plt.title(spFile.GetChannelTitle(channel_index))
    plt.ylim([d_min, d_max])
    plt.ylabel(spFile.GetChannelUnits(channel_index))
    if rescale:
        plt.plot(np.linspace(start, end, i_end - i_start), data[i_start:i_end])
    else:
        plt.plot(data[i_start:i_end])
    plt.show()

In [ ]:
for i in [67, 70, 71, 72]:
    plot(file_recording, i, rescale=True)

In [ ]:
# for i in range(spFile.MaxChannels()):
# print(i, ""  else spFile.GetChannelTitle(i), spFile.ChannelType(i))

In [ ]:
source_data = dict(
    CEDRecording=dict(file_path=file_recording),
    CEDStimulus=dict(file_path=file_recording),
)

In [ ]:
converter = mease_lab_to_nwb.CEDNWBConverter(source_data=source_data)

In [ ]:
print(source_data)

In [ ]:
# Get metadata from source data and modify any values you want
metadata = converter.get_metadata()

# Get conversion options and modify any values you want
conversion_options = converter.get_conversion_options()

# OPTIONAL - Validate source_data, metadata and conversion_options
converter.validate_source(source_data)
converter.validate_metadata(metadata)
converter.validate_conversion_options(conversion_options)

# Run conversion
output_file = "out_example_ced.nwb"

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    save_to_file=True,
    overwrite=True,
    conversion_options=conversion_options,
)

In [ ]:
output_file = "out_example_ced.nwb"
io = NWBHDF5IO(output_file, "r")
nwbfile = io.read()
nwb2widget(nwbfile)

In [ ]:
# io.close()